In [ ]:
import os
import io
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LSTM, Bidirectional, Conv1D, MaxPooling1D,
    Flatten, Multiply, Permute, Concatenate, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D, RepeatVector, Add
)
from tensorflow.keras.callbacks import EarlyStopping

from ipywidgets import widgets, VBox, HBox, Output, Label, Button, FileUpload, Dropdown, IntSlider, DatePicker
from IPython.display import display, clear_output



# --------------------------
# Funkcja do wczytania i przygotowania danych
def load_and_process_data(file_path_or_buffer):
    if isinstance(file_path_or_buffer, str):
        df = pd.read_csv(file_path_or_buffer, parse_dates=["timestamp"])
    else:
        df = pd.read_csv(io.BytesIO(file_path_or_buffer), parse_dates=["timestamp"])
    df = df.sort_values("timestamp")
    return df

# --------------------------
# Architektury modeli
#stacked LSTM (sunny et.al)
def create_model_1(input_shape):
    model = Sequential([
        LSTM(128, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(0.3),
        LSTM(128, activation='relu'),
        Dense(1)
    ])
    return model


#istiake sunny
def create_model_2(input_shape):
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape),
        Dropout(0.3),
        Bidirectional(LSTM(64)),
        Dense(16, activation='relu'),
        Dense(1)
    ])
    return model




def attention_3d_block(inputs, single_attention_vector=False):
    time_steps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    a = Permute((2, 1))(inputs)
    a = Dense(time_steps, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1))(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

def create_model_3(input_shape):
    inputs = Input(shape=input_shape)

    # Warstwa konwolucyjna
    x = Conv1D(filters=64, kernel_size=1, activation='relu')(inputs)
    x = Dropout(0.3)(x)

    # Dwukierunkowy LSTM
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.3)(x)

    # Blok uwagi
    x = attention_3d_block(x)

    # Spłaszczamy wynik
    x = Flatten()(x)

    # Wyjście modelu
    outputs = Dense(1)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def add_positional_encoding(x):
    seq_len = tf.shape(x)[1]
    d_model = tf.shape(x)[2]

    pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
    i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
    angle_rates = 1 / tf.pow(10000.0, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    angle_rads = pos * angle_rates

    sines = tf.sin(angle_rads[:, 0::2])
    cosines = tf.cos(angle_rads[:, 1::2])
    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = tf.expand_dims(pos_encoding, 0)
    return x + pos_encoding

def transformer_block(x, head_size, num_heads, ff_dim, dropout=0.1):
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    attn_output = Dropout(dropout)(attn_output)
    x = Add()([x, attn_output])
    x = LayerNormalization(epsilon=1e-6)(x)

    ffn = Dense(ff_dim, activation="relu")(x)
    ffn = Dense(x.shape[-1])(ffn)
    ffn = Dropout(dropout)(ffn)
    x = Add()([x, ffn])
    x = LayerNormalization(epsilon=1e-6)(x)
    return x

def create_model_4(input_shape):
    inputs = Input(shape=input_shape)

    # Dense encoding (jak w PyTorch Linear encoder)
    x_encoded = Dense(64)(inputs)

    # LSTM blok
    x_lstm_pos = tf.keras.layers.Lambda(add_positional_encoding)(x_encoded)
    x_lstm = LSTM(32, return_sequences=True)(x_lstm_pos)
    x_lstm_last = x_lstm[:, -1, :]  # ostatni krok czasowy

    # Transformer blok
    x_trans_pos = tf.keras.layers.Lambda(add_positional_encoding)(x_encoded)
    x_trans = transformer_block(x_trans_pos, head_size=64, num_heads=4, ff_dim=128, dropout=0.1)
    x_trans_last = x_trans[:, -1, :]  # ostatni krok czasowy

    # Połączenie LSTM + Transformer
    x_combined = Concatenate()([x_lstm_last, x_trans_last])
    x = Dense(64, activation="relu")(x_combined)
    x = Dropout(0.2)(x)
    outputs = Dense(1)(x)

    model = Model(inputs, outputs)
    return model



architecture_dict = {
    "LSTM 64": create_model_1,
    "Bidirectional LSTM": create_model_2,
    "Bidirectional LSTM + Attention": create_model_3,
    "LSTM + Transformer": create_model_4,
}

# --------------------------
# UI i widgety
output = Output()

upload = FileUpload(
    accept=".csv",
    multiple=False,
    description="Wgraj CSV z danymi"
)

choice_asset = Dropdown(
    options=["BTC", "ETH", "SP500", "AAPL"],
    description="Przewiduj:",
    value="ETH"
)

choice_mode = Dropdown(
    options=["Trenuj od zera", "Fine-tuning istniejącego modelu"],
    description="Tryb uczenia:",
    value="Fine-tuning istniejącego modelu"
)

# Usuwamy suwak train_test_split i zastępujemy go DatePickerem
train_test_date_picker = DatePicker(
    description='Data podziału:',
    disabled=False
)

choice_architecture = Dropdown(
    options=list(architecture_dict.keys()),
    description="Architektura:",
    value="LSTM 64"
)

epochs_slider = IntSlider(
    value=15,
    min=1,
    max=150,
    step=1,
    description="Epoki:"
)

button_run = Button(
    description="Uruchom trening i predykcję",
    button_style='success'
)

save_model_checkbox = widgets.Checkbox(
    value=False,
    description='Zapisz model po treningu'
)

model_name_text = widgets.Text(
    value='nowy_model',
    description='Nazwa pliku:',
    placeholder='np. LSTM_ETH_model',
    disabled=False
)

existing_model_name_text = widgets.Text(
    value='./models/LSTM_model.h5',
    description='Model do fine-tuningu:',
    placeholder='ścieżka do modelu .h5',
    disabled=False
)


default_data_path = "./data/data.csv"
default_model_path = "./models/LSTM_model.h5"

# --------------------------
# Funkcja dopasowania wag Dense z modelu starego do nowego, jeśli kształt pasuje
def try_copy_dense_weights(model_new, model_old):
    dense_new = model_new.layers[-1]
    dense_old = model_old.layers[-1]
    weights_old = dense_old.get_weights()
    weights_new = dense_new.get_weights()

    print(f"🔍 Kształt wag starego modelu (Dense): {weights_old[0].shape}, bias: {weights_old[1].shape}")
    print(f"🔍 Kształt wag nowego modelu (Dense): {weights_new[0].shape}, bias: {weights_new[1].shape}")

    if weights_old[0].shape == weights_new[0].shape and weights_old[1].shape == weights_new[1].shape:
        dense_new.set_weights(weights_old)
        print("⚡️ Skopiowano wagi Dense z modelu starego.")
    else:
        print("⚠️ Kształt wag Dense nie pasuje. Pomijam kopiowanie wag Dense.")


def try_copy_dense_weights_adapted(model_new, model_old):
    dense_new = model_new.layers[-1]
    dense_old = model_old.layers[-1]
    weights_old = dense_old.get_weights()
    weights_new = dense_new.get_weights()

    w_old, b_old = weights_old
    w_new, b_new = weights_new

    print(f"🔍 Stary Dense weights shape: {w_old.shape}, bias: {b_old.shape}")
    print(f"🔍 Nowy Dense weights shape: {w_new.shape}, bias: {b_new.shape}")

    # Dopasuj kształt wag wagi przez obcięcie lub dopełnienie zerami
    w_new_adapted = np.zeros_like(w_new)
    b_new_adapted = np.zeros_like(b_new)

    min_rows = min(w_old.shape[0], w_new.shape[0])
    min_cols = min(w_old.shape[1], w_new.shape[1])
    min_bias = min(b_old.shape[0], b_new.shape[0])

    # Kopiuj część wspólną wag
    w_new_adapted[:min_rows, :min_cols] = w_old[:min_rows, :min_cols]
    b_new_adapted[:min_bias] = b_old[:min_bias]

    dense_new.set_weights([w_new_adapted, b_new_adapted])
    print("⚡️ Wagi Dense zostały dopasowane i skopiowane.")


# --------------------------
# Funkcja wyciągająca input_shape z modelu starego (seq_len i features)
def get_input_shape_from_model(model_old):
    for layer in model_old.layers:
        if hasattr(layer, 'input_shape') and layer.input_shape is not None:
            # Kształt wejścia to (None, seq_len, features)
            input_shape = layer.input_shape[1:]  # pomijamy None
            return input_shape
    return None

# --------------------------
# Funkcja obsługująca trening i predykcję
def on_button_run_clicked(b):
    with output:
        clear_output()
        print("Wczytywanie danych... ⏳")
        try:
            if upload.value:
                for filename in upload.value:
                    content = upload.value[filename]["content"]
                    df = load_and_process_data(content)
                    print(f"Wczytano z wgranego pliku: {filename}")
                    break
            else:
                df = load_and_process_data(default_data_path)
                print(f"Wczytano domyślny plik: {default_data_path}")
        except Exception as e:
            print(f"Błąd wczytywania danych: {e}")
            return

        asset = choice_asset.value
        print(f"Wybrany asset do predykcji: {asset}")
        if asset not in df.columns:
            print(f"Brak kolumny {asset} w danych!")
            return

        # Sprawdzamy, czy wybrano datę podziału
        split_date = train_test_date_picker.value
        if split_date is None:
            print("❌ Proszę wybrać datę podziału danych!")
            return

        df_clean = df.dropna(subset=[asset])

        # Ustawiamy datę podziału jako datetime (w razie czego)
        split_date = pd.to_datetime(split_date)
        print(f"Data podziału na train/test: {split_date.strftime('%Y-%m-%d %H:%M:%S')}")

        # Skalowanie całego zbioru przed podziałem (ważne)
        data_single = df_clean[[asset]]
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(data_single.values)

        # Domyślny seq_len
        default_seq_len = 24
        seq_len = default_seq_len

        # Jeśli fine-tuning, spróbuj pobrać seq_len z modelu
        if choice_mode.value == "Fine-tuning istniejącego modelu":
            try:
                model_path = existing_model_name_text.value.strip()
                if not os.path.isfile(model_path):
                    print(f"❌ Plik modelu '{model_path}' nie istnieje!")
                    return
                model_old = load_model(model_path, compile=False)
                print(f"Wczytano model do fine-tuningu z {model_path}")
                input_shape_old = get_input_shape_from_model(model_old)
                if input_shape_old is not None:
                    seq_len = input_shape_old[0]
                    features_old = input_shape_old[1]
                    print(f"Input shape z modelu starego: {input_shape_old} (seq_len={seq_len}, features={features_old})")
                else:
                    print("Nie udało się wyciągnąć input_shape ze starego modelu, używam domyślnego seq_len.")
                    seq_len = default_seq_len
            except Exception as e:
                print(f"Błąd przy wczytywaniu starego modelu: {e}")
                seq_len = default_seq_len

        # Tworzymy sekwencje X, y dla całego zbioru
        X_all, y_all, timestamps_all = [], [], []
        for i in range(len(scaled_data) - seq_len):
            X_all.append(scaled_data[i:i+seq_len])
            y_all.append(scaled_data[i+seq_len, 0])
            timestamps_all.append(df_clean["timestamp"].iloc[i + seq_len])

        X_all = np.array(X_all)
        y_all = np.array(y_all)
        timestamps_all = pd.Series(timestamps_all)

        # Podział na train/test wg daty - timestampy w timestamps_all to timestampy odpowiadające y
        mask_train = timestamps_all <= split_date
        mask_test = timestamps_all > split_date

        X_train = X_all[mask_train.values]
        y_train = y_all[mask_train.values]
        X_test = X_all[mask_test.values]
        y_test = y_all[mask_test.values]
        timestamps_test = timestamps_all[mask_test.values]

        print(f"Dane podzielone na trening ({len(X_train)}) i test ({len(X_test)})")

        if len(X_train) == 0 or len(X_test) == 0:
            print("❌ Zbyt mało danych w zbiorze treningowym lub testowym po podziale datą. Proszę wybrać inną datę.")
            return

        input_shape_new = (seq_len, 1)  # mamy jeden feature

        # Tworzymy nowy model i kopiujemy wagi Dense jeśli możliwe
        if choice_mode.value == "Fine-tuning istniejącego modelu":
            try:
                create_fn = architecture_dict[choice_architecture.value]
                model = create_fn(input_shape_new)
                try_copy_dense_weights_adapted(model, model_old)
            except Exception as e:
                print(f"Błąd przy tworzeniu nowego modelu lub kopiowaniu wag: {e}")
                print("Tworzę model od zera z domyślnym shape.")
                create_fn = architecture_dict[choice_architecture.value]
                model = create_fn(input_shape_new)
        else:
            print("Tworzenie modelu od zera z domyślnym shape...")
            create_fn = architecture_dict[choice_architecture.value]
            model = create_fn(input_shape_new)

        model.compile(optimizer="adam", loss="mse", metrics=["mae"])

        early_stopping = EarlyStopping(monitor="val_loss", patience=12)

        print("Start treningu...")
        history = model.fit(
            X_train, y_train,
            validation_split=0.1,
            epochs=epochs_slider.value,
            batch_size=32,
            callbacks=[early_stopping],
            verbose=1
        )

        if save_model_checkbox.value:
            model.save(f"{model_name_text.value}.h5")
            print(f"Model zapisany jako {model_name_text.value}.h5")

        print("Robię predykcje na zbiorze testowym...")
        pred_test_scaled = model.predict(X_test)

        # Odskalowanie do oryginalnych wartości
        y_test_orig = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()
        pred_test_orig = scaler.inverse_transform(pred_test_scaled).flatten()



        # Obliczenie miar regresji
        mse = mean_squared_error(y_test_orig, pred_test_orig)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test_orig, pred_test_orig)
        mape = np.mean(np.abs((y_test_orig - pred_test_orig) / y_test_orig)) * 100
        r2 = r2_score(y_test_orig, pred_test_orig)

        # Wyświetlenie metryk
        print("\n📊 Miary regresji:")
        print(f"➡️  MSE  (Mean Squared Error): {mse:.4f}")
        print(f"➡️  RMSE (Root Mean Squared Error): {rmse:.4f}")
        print(f"➡️  MAE  (Mean Absolute Error): {mae:.4f}")
        print(f"➡️  MAPE (Mean Absolute Percentage Error): {mape:.2f}%")
        print(f"➡️  R²   (R-squared): {r2:.4f}")

        # Wykres predykcji i rzeczywistych wartości z osi czasu z timestamps_test
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=timestamps_test,
            y=y_test_orig,
            mode='lines',
            name='Rzeczywiste'
        ))
        fig.add_trace(go.Scatter(
            x=timestamps_test,
            y=pred_test_orig,
            mode='lines',
            name='Predykcja'
        ))
        fig.update_layout(
            title=f"Predykcja na zbiorze testowym ({asset})",
            xaxis_title="Data",
            yaxis_title="Wartość",
            hovermode='x unified'
        )
        fig.show()

# --------------------------
# Podpinanie eventu do przycisku
button_run.on_click(on_button_run_clicked)

# --------------------------
# Układ widgetów
ui_left = VBox([
    upload,
    choice_asset,
    train_test_date_picker,
    choice_mode,
    existing_model_name_text,
    choice_architecture,
    epochs_slider,
    save_model_checkbox,
    model_name_text,
    button_run
])

ui = HBox([ui_left])

display(ui, output)


Output()

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import load_model
from ipywidgets import widgets, VBox, DatePicker, Text, Button, Dropdown, Output
from IPython.display import display, clear_output

# Funkcja wczytuje dane i sortuje je po dacie
def load_and_prepare_data(path='./data/data.csv'):
    df = pd.read_csv(path, parse_dates=['timestamp'])
    df = df.sort_values('timestamp')
    return df

# Pobiera kształt wejścia modelu (potrzebny do przygotowania sekwencji)
def get_input_shape_from_model(model):
    try:
        shape = model.input_shape
        if isinstance(shape, list):
            shape = shape[0]
        return shape[1], shape[2]
    except Exception:
        return None

# Tworzy interfejs użytkownika do wczytania modelu i uruchomienia predykcji
def create_prediction_ui():
    output = Output()

    model_file_text = Text(
        value='./models/ETH_100_3.h5',
        description='Model .h5:',
        layout=widgets.Layout(width='60%')
    )

    date_picker = DatePicker(
        description='Data testu od:',
        disabled=False
    )

    button_run = Button(
        description="Uruchom predykcję",
        button_style='success'
    )

    asset_dropdown = Dropdown(
        options=[],
        description='Cecha:',
        disabled=False
    )

    # Wczytanie listy dostępnych cech (kolumn) z pliku danych
    def update_asset_options():
        try:
            df = load_and_prepare_data()
            cols = [col for col in df.columns if col != 'timestamp']
            asset_dropdown.options = cols
            if cols:
                asset_dropdown.value = cols[0]
        except Exception:
            asset_dropdown.options = []

    update_asset_options()

    # Główna funkcja uruchamiana po kliknięciu przycisku
    def on_button_run_clicked(b):
        with output:
            clear_output()
            print("Wczytywanie modelu i danych...")
            model_path = model_file_text.value.strip()
            if not model_path:
                print("❌ Podaj ścieżkę do pliku modelu!")
                return

            # Próba wczytania modelu
            try:
                model = load_model(model_path, compile=False)
                print(f"Wczytano model z: {model_path}")
            except Exception as e:
                print(f"❌ Błąd wczytywania modelu: {e}")
                return

            # Próba wczytania danych
            try:
                df = load_and_prepare_data()
                print("Wczytano dane z './data/data.csv'")
            except Exception as e:
                print(f"❌ Błąd wczytywania danych: {e}")
                return

            asset = asset_dropdown.value
            if asset is None:
                print("❌ Proszę wybrać cechę do predykcji!")
                return

            split_date = date_picker.value
            if split_date is None:
                print("❌ Proszę wybrać datę testową!")
                return
            split_date = pd.to_datetime(split_date)

            # Przygotowanie danych: czyszczenie i skalowanie
            df_clean = df.dropna(subset=[asset])
            scaler = MinMaxScaler()
            scaled_data = scaler.fit_transform(df_clean[[asset]].values)

            # Sprawdzenie kształtu wejściowego modelu
            input_shape = get_input_shape_from_model(model)
            if input_shape is None:
                print("❌ Nie udało się pobrać input_shape!")
                return

            seq_len, features = input_shape

            if len(scaled_data) <= seq_len:
                print(f"❌ Za mało danych ({len(scaled_data)}) na długość sekwencji ({seq_len})")
                return

            # Tworzenie sekwencji dla predykcji
            X_all, y_all, timestamps_all = [], [], []
            for i in range(len(scaled_data) - seq_len):
                seq = scaled_data[i:i + seq_len]
                if features == 1:
                    X_all.append(seq)
                else:
                    # Dopasowanie liczby cech jeśli model oczekuje więcej niż 1 feature
                    repeated = np.repeat(seq, features, axis=1)[:, :features]
                    X_all.append(repeated)
                y_all.append(scaled_data[i + seq_len, 0])
                timestamps_all.append(df_clean['timestamp'].iloc[i + seq_len])

            X_all = np.array(X_all)
            y_all = np.array(y_all)
            timestamps_all = pd.Series(timestamps_all)

            # Podział na zbiór testowy według wybranej daty
            mask_test = timestamps_all >= split_date
            X_test = X_all[mask_test.values]
            y_test = y_all[mask_test.values]
            timestamps_test = timestamps_all[mask_test.values]

            if len(X_test) == 0:
                print("❌ Brak danych testowych po tej dacie.")
                return

            print(f"Zbiór testowy: {len(X_test)} próbek. Robię predykcję...")
            pred_test_scaled = model.predict(X_test)

            # Odwrócenie skalowania do oryginalnych wartości
            y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
            pred_test_orig = scaler.inverse_transform(pred_test_scaled).flatten()

            # Obliczenie metryk jakości predykcji
            mse = mean_squared_error(y_test_orig, pred_test_orig)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_test_orig, pred_test_orig)
            mape = np.mean(np.abs((y_test_orig - pred_test_orig) / y_test_orig)) * 100
            r2 = r2_score(y_test_orig, pred_test_orig)

            print("\n📊 Miary regresji:")
            print(f"MSE:  {mse:.4f}")
            print(f"RMSE: {rmse:.4f}")
            print(f"MAE:  {mae:.4f}")
            print(f"MAPE: {mape:.2f}%")
            print(f"R²:   {r2:.4f}")

            # Rysowanie wykresu wyników
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=timestamps_test, y=y_test_orig, mode='lines', name='Rzeczywiste'))
            fig.add_trace(go.Scatter(x=timestamps_test, y=pred_test_orig, mode='lines', name='Predykcja'))
            fig.update_layout(
                title=f"Predykcja od {split_date.date()}",
                xaxis_title="Data",
                yaxis_title="Wartość",
                hovermode='x unified'
            )
            fig.show()

    button_run.on_click(on_button_run_clicked)

    ui = VBox([model_file_text, asset_dropdown, date_picker, button_run, output])
    display(ui)


In [ ]:
create_prediction_ui()


In [ ]:
create_prediction_ui()


In [ ]:
create_prediction_ui()


In [ ]:
create_prediction_ui()


In [ ]:
create_prediction_ui()

In [ ]:
create_prediction_ui()

In [ ]:
create_prediction_ui()